In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler, select_urls
from IPython.display import display
from IPython.core.debugger import set_trace

In [2]:
crawler = NewsCrawler()#'vice')

# 1. URL을 모은다

In [3]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [4]:
collect_results = crawler.collect(); display(*collect_results)

100.00% (408.16 seconds): Collecting URLs... usatoday            

n_total     25606
n_unique    25543
dtype: int64

,pubs,actual_pub
https://businessinsider.co.za/anne-sacoolas-diplomat-husband-immunity-unclear-expert-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/asylum-seekers-get-access-to-uif-with-labour-department-regulation-update-2019-10,"businessinsider, news24",businessinsider
https://businessinsider.co.za/saudi-arabia-laws-tourists-pitfalls-2019-10,"businessinsider, news24",businessinsider
https://hotair.com/archives/ed-morrissey/2019/10/09/hmmm-durham-expanding-scope-operation-crossfire-hurricane-probe-include-mueller,"townhall, hotair",hotair
https://msnbc.com/morning-joe/watch/congressman-on-why-he-s-threatening-pompeo-s-salary-70923845715,"msnbc, nbcnews",msnbc
https://msnbc.com/morning-joe/watch/joe-obstructing-an-inquiry-likely-to-further-damage-trump-s-standing-70912069958,"msnbc, nbcnews",msnbc
https://msnbc.com/morning-joe/watch/trump-being-used-by-giuliani-says-conservative-writer-70913093972,"msnbc, nbcnews",msnbc
https://msnbc.com/morning-joe/watch/what-is-the-legal-remedy-for-trump-s-refusal-to-cooperate-70908997891,"msnbc, nbcnews",msnbc
https://msnbc.com/msnbc/watch/watch-courtney-kube-s-son-wanders-on-set-during-live-reporting-70918725778,"msnbc, nbcnews",msnbc
https://msnbc.com/the-beat-with-ari/watch/swizz-beatz-on-working-with-beyonce-jay-z-drake-and-upending-the-art-world-70266437589,"msnbc, nbcnews",msnbc


huffpost                  69
cnn                      801
investing.com            964
politico                 216
time                      17
cnbc                     160
fox                      324
bbc                      285
businessinsider          541
morningstar               89
wsj                      102
nytimes                  105
guardian                 162
reuters                 1143
washingtontimes          463
washingtonpost           127
cbs                      409
marketwatch              149
atlantic                 102
vice                      97
npr                      880
newrepublic               20
yahoo                    420
independent              704
heritage                 286
zdnet                    392
townhall                 629
abcnews                   95
hotair                    84
cbc                      292
                        ... 
metro                    252
msnbc                    108
nationalreview           726
news24        

In [5]:
collect_results[2]['thinkprogress']

82

# 2. 모은 URL중 이미 받은 것은 버린다

In [6]:
select_results = crawler.select(); display(*select_results)

100.00% (59.76 seconds): Selecting URLs... cryptonews          

n_total     3742
n_unique    3742
dtype: int64

,pubs,actual_pub


huffpost                  9
cnn                      66
investing.com           172
politico                 11
time                      6
cnbc                     36
fox                     114
bbc                      25
businessinsider         179
morningstar              16
wsj                      33
nytimes                  18
guardian                 44
reuters                 348
washingtontimes          65
washingtonpost           58
cbs                      33
marketwatch              37
atlantic                  6
vice                     37
npr                      16
newrepublic               1
yahoo                   221
independent              75
heritage                  1
zdnet                    34
townhall                 31
abcnews                  25
hotair                   36
cbc                      41
                       ... 
metro                    80
msnbc                    20
nationalreview           35
news24                  126
techcrunch          

In [7]:
crawler.urls_selected['thinkprogress']

set()

# 3. 해당 URL의 뉴스를 다운로드한다

In [8]:
download_results = crawler.download(); download_results

100.00% (3663.16 seconds): Downloading... dailymail           

,downloaded,trashed
abcnews,24,1
afr,37,0
aljazeera,14,74
americanconservative,8,1
arirang,0,0
arstechnica,4,0
arynews,9,20
atlantic,6,0
axios,56,28
bbc,22,3


In [20]:
import hashlib
[hashlib.sha1(url.encode('utf-8')).hexdigest() for url in crawler.urls_final['japan-news']['trashed']]

['d80494a4761ead4276e510dc4f64762e319902e0',
 'f25b1d06e58c05786ad57fe9bea9817859de3861',
 '02d668060004e25c2299c9a23445e38a9c6ec771',
 '2d3e330325b35d2c622806feac055478a4a61559',
 '2e87c621d1ffb7f28485496eb82814891ff5dd1b',
 '40cf0a5a2b87e040cd211da1855cb615702cb0ca',
 'feefa98b143ce76ef3cadf0a2764ca634f773fee',
 '0a900188f145b48aad8c70bde741aea563dba542',
 'ef7e33fa30c8be4b6d90681b383234ee4f73e51a',
 '64cc9a5e5a51520306154fda70d482b57a7420b0',
 '12e43a97340dc5a9f9270b38a9705e461456041b',
 'd71b4a23216b50deb276d3ccd8c1905939f5a9c2',
 '9a5e5f813637acd5359da986e562fa9ad9858f4b',
 '44c36bff1fe4d64fb468131f1221959fa30718d6',
 'dbf61f3dd462f9c650d0b2f42ca69717701fc7d1',
 '385dbc7ad54410ac2078055044e34c6e30cfde4b',
 'a31fe291585e194540805022e45df9ebdbabc9fc',
 '66afae226b2dc5d329fc731f7931f9f6d6f37e23']